# Modelamiento

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.MLDS6_Assets.preprocessing import countFilesinFolder
from src.MLDS6_Assets.visualization import plotDataPartition
from src.MLDS6_Assets.models import trainTestSplitImages

## Partición de datos

Para la partición de datos, vamos a tomar de base los datos unidos y balanceados en un único dataset. Para realizar la partición, usaremos la función train test split de sklearn y mantener el esquema de carpetas

## Selección y diseño de modelos

Para abordar el problema de clasificación de imágenes en el diagnóstico de neumonía a partir de radiografías, hemos seleccionado los modelos VGG16 y EfficientNetB0, ambos disponibles en TensorFlow. Esta elección se justifica debido a que ambos modelos son redes neuronales convolucionales bien probadas en tareas de reconocimiento de imágenes y permiten capturar características visuales esenciales en radiografías. VGG16, con su arquitectura profunda y secuencial, es eficaz en detectar patrones sutiles y texturas asociadas a los signos de neumonía, como opacidades pulmonares; mientras que EfficientNetB0 se distingue por su capacidad de procesamiento optimizada y eficiente, lo que es ideal para implementaciones en tiempo real sin comprometer precisión. Además, se implementará transfer learning utilizando modelos preentrenados, lo que permitirá aprovechar características previamente aprendidas en grandes conjuntos de datos, acelerando el entrenamiento y mejorando la generalización en nuestro conjunto de datos específico. Al combinar ambos modelos, podemos lograr un balance entre precisión y velocidad, mejorando la fiabilidad diagnóstica. Además, ambos modelos se pueden adaptar añadiendo capas densas y técnicas como dropout o normalización para ajustarse mejor a las necesidades específicas del conjunto de datos, maximizando así el rendimiento en la tarea de clasificación.

Adicionalmente, las imagenes deben redimensionarse para tener una dimensión esprcifica para el modelo

### Estandarizacion de los datos
Teniendo en cuenta que los datos se encuentran en un tamaño no apto para el entrenamiento se busca estandarizarlo para que todo funcione bien

In [ ]:
# Ruta a la carpeta con las imágenes
folder_path = "MLDS2_Project_Data_Split/test/virusPneumonia"

# Inicializar variables para el cálculo del promedio
aspect_ratios = []

# Iterar sobre cada archivo en la carpeta
for filename in os.listdir(folder_path):
    image_path = os.path.join(folder_path, filename)
    try:
        with Image.open(image_path) as img:
            width, height = img.size
            aspect_ratio = width / height
            aspect_ratios.append(aspect_ratio)
    except Exception as e:
        print(f"Error al procesar {filename}: {e}")

# Calcular el promedio de la relación de aspecto
if aspect_ratios:
    average_aspect_ratio = sum(aspect_ratios) / len(aspect_ratios)
    print(f"Promedio de la relación de aspecto: {average_aspect_ratio}")
else:
    print("No se encontraron imágenes para calcular la relación de aspecto.")